In [ ]:
from langchain.tools import Tool
from langchain.utilities.wikipedia import WikipediaAPIWrapper
from langchain.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain.document_loaders.web_base import WebBaseLoader
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()

def save_to_txt(text: str, filename: str = "research.txt"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(text)
    return f"Research saved to {filename}"

wiki_tool = Tool(
    name="Wikipedia",
    func=WikipediaAPIWrapper().run,
    description="Searches and returns results from Wikipedia"
)
duck_tool = Tool(
    name="DuckDuckGo",
    func=DuckDuckGoSearchRun().run,
    description="Searches and returns results from DuckDuckGo"
)
def web_scrape(url: str) -> str:
    try:
        # User-Agent 추가
        session = requests.Session()
        session.headers.update({
            "User-Agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"
        })
        loader = WebBaseLoader(url, session=session)
        docs = loader.load()
        return docs[0].page_content[:2000]
    except Exception as e:
        return f"Failed to load content from {url}: {str(e)}"
web_tool = Tool(
    name="WebScraper",
    func=web_scrape,
    description="Extracts and returns main text from a website URL"
)
save_tool = Tool(
    name="SaveToTxt",
    func=save_to_txt,
    description="Saves research content to a .txt file"
)

llm = ChatOpenAI(temperature=0, model="gpt-4")

tools = [wiki_tool, duck_tool, web_tool, save_tool]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

query = "Research about the XZ backdoor"
result = agent.run(query)
print(result)




> Entering new AgentExecutor chain...
I should start by searching for information about the XZ backdoor to gather more details.
Action: DuckDuckGo
Action Input: XZ backdoor

HTTPError: 